In [88]:
import pandas as pd
import numpy as np
import pandas_datareader.data as web
import datetime
import sqlite3
import os
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS
import seaborn
import statistics

pd.set_option('mode.chained_assignment',  None) # 경고 off

In [89]:
diff = pd.read_csv('domi.csv', index_col=0, encoding='CP949')
AR = pd.read_csv('AR.csv', index_col=0, encoding='CP949')
data = pd.read_csv('data_diff.csv', index_col=0, encoding='CP949')
price = pd.read_csv('domi_price.csv', index_col=0, encoding='CP949')
data = data[['SP500']]

In [90]:
###### ProxyA_up
k=0
A=pd.DataFrame()
for n in range(0, len(price.columns)):
    for m in range(35,len(diff)-20):
        if(diff.iloc[m,n]>=0.1 and diff.iloc[m,n]<0.5):
            if(pd.isnull(diff.iloc[m-35:m+11, n]).sum()==0):
                A.loc[k,"name"]=diff.columns[n]
                A.loc[k, "day"]=pd.to_datetime(diff.index[m]).date()
                A.loc[k,"diff"]=diff.iloc[m,n]
                A.loc[k,"std"]=statistics.stdev(diff.iloc[m-30:m,n]) 
                A.loc[k,"SP500"]=data.iloc[m,0]
                if(A.loc[k,"SP500"]>0):
                    A.loc[k,"MR0_dum"]=1
                else:
                    A.loc[k,"MR0_dum"]=0
                A.loc[k, "lastdiff"]=diff.iloc[m-1,n]
                A.loc[k, "lastprice"]=price.iloc[m-1,n]
                A.loc[k, "52_max"]=max(price.iloc[m-30:m,n])
                A.loc[k, "52_min"]=min(price.iloc[m-30:m,n])
                A.loc[k, "52_WK_HI"]=price.iloc[m-1,n]/max(price.iloc[m-30:m,n])
                A.loc[k, "52_WK_LO"]=min(price.iloc[m-30:m,n])/price.iloc[m-1,n]
                B=pd.DataFrame()
                B.index.names = ['Date']
                B["diff"]=diff.iloc[m-30:m+11,n]
                B["SP500"]=data.iloc[m-30:m+11,0]                
                B.to_excel('Aup/'+diff.columns[n]+'-'+str(pd.to_datetime(diff.index[m]).date())+'.xlsx', index=True)
                roll_reg2 = RollingOLS.from_formula('diff ~ SP500', window=26,min_nobs=26, data=B,missing='drop') #capm모델용 무위험이자율 하기 다요인용3
                model3 = roll_reg2.fit()
                para2 = model3.params
                a=para2.iloc[25,0]
                b=para2.iloc[25,1]
                C=1+(B.iloc[25:41,0]-(a+b*B.iloc[25:41,1]))
                A.loc[k, "CAR(-5,-1)"]=C[0]*C[1]*C[2]*C[3]*C[4]-1
                if(A.loc[k, "CAR(-5,-1)"]<0):
                        A.loc[k, "CONTRADICTION"]=1
                else:
                        A.loc[k, "CONTRADICTION"]=0
                A.loc[k, "Day1"] = C[6]-1
                A.loc[k, "Day2"] = C[7]-1
                ppp=1
                for pp in range(6, 11):
                    ppp = ppp*C[pp]
                A.loc[k, "Day5"] = ppp-1
                ppp=1
                for pp in range(6, 16):
                    ppp = ppp*C[pp]
                A.loc[k, "Day10"] = ppp-1
                C = 1+AR.iloc[m-5:m+11, n]
                A.loc[k, "Deep_Day1"] = C[6]-1
                A.loc[k, "Deep_Day2"] = C[7]-1
                ppp=1
                for pp in range(6, 11):
                    ppp = ppp*C[pp]
                A.loc[k, "Deep_Day5"] = ppp-1
                ppp=1
                for pp in range(6, 16):
                    ppp = ppp*C[pp]
                A.loc[k, "Deep_Day10"] = ppp-1
                k=k+1

            

A.to_excel('ProxyA_up.xlsx', index=False)

In [91]:
###### ProxyA_down
k=0
A=pd.DataFrame()
for n in range(0, len(price.columns)):
    for m in range(35,len(diff)-20):
        if(diff.iloc[m,n]<=-0.1 and diff.iloc[m,n]>-0.5):
            if(pd.isnull(diff.iloc[m-35:m+11, n]).sum()==0):
                A.loc[k,"name"]=diff.columns[n]
                A.loc[k, "day"]=pd.to_datetime(diff.index[m]).date()
                A.loc[k,"diff"]=diff.iloc[m,n]
                A.loc[k,"std"]=statistics.stdev(diff.iloc[m-30:m,n]) 
                A.loc[k,"SP500"]=data.iloc[m,0]
                if(A.loc[k,"SP500"]>0):
                    A.loc[k,"MR0_dum"]=1
                else:
                    A.loc[k,"MR0_dum"]=0
                A.loc[k, "lastdiff"]=diff.iloc[m-1,n]
                A.loc[k, "lastprice"]=price.iloc[m-1,n]
                A.loc[k, "52_max"]=max(price.iloc[m-30:m,n])
                A.loc[k, "52_min"]=min(price.iloc[m-30:m,n])
                A.loc[k, "52_WK_HI"]=price.iloc[m-1,n]/max(price.iloc[m-30:m,n])
                A.loc[k, "52_WK_LO"]=min(price.iloc[m-30:m,n])/price.iloc[m-1,n]
                B=pd.DataFrame()
                B.index.names = ['Date']
                B["diff"]=diff.iloc[m-30:m+11,n]                
                B["SP500"]=data.iloc[m-30:m+11,0]           
                B.to_excel('Adown/'+diff.columns[n]+'-'+str(pd.to_datetime(diff.index[m]).date())+'.xlsx', index=True)
                roll_reg2 = RollingOLS.from_formula('diff ~ SP500', window=26,min_nobs=26, data=B,missing='drop') #capm모델용 무위험이자율 하기 다요인용3
                model3 = roll_reg2.fit()
                para2 = model3.params
                a=para2.iloc[25,0]
                b=para2.iloc[25,1]
                C=1+(B.iloc[25:41,0]-(a+b*B.iloc[25:41,1]))
                A.loc[k, "CAR(-5,-1)"]=C[0]*C[1]*C[2]*C[3]*C[4]-1
                if(A.loc[k, "CAR(-5,-1)"]<0):
                        A.loc[k, "CONTRADICTION"]=1
                else:
                        A.loc[k, "CONTRADICTION"]=0
                A.loc[k, "Day1"] = C[6]-1
                A.loc[k, "Day2"] = C[7]-1
                ppp=1
                for pp in range(6, 11):
                    ppp = ppp*C[pp]
                A.loc[k, "Day5"] = ppp-1
                ppp=1
                for pp in range(6, 16):
                    ppp = ppp*C[pp]
                A.loc[k, "Day10"] = ppp-1
                C = 1+AR.iloc[m-5:m+11, n]
                A.loc[k, "Deep_Day1"] = C[6]-1
                A.loc[k, "Deep_Day2"] = C[7]-1
                ppp=1
                for pp in range(6, 11):
                    ppp = ppp*C[pp]
                A.loc[k, "Deep_Day5"] = ppp-1
                ppp=1
                for pp in range(6, 16):
                    ppp = ppp*C[pp]
                A.loc[k, "Deep_Day10"] = ppp-1
                k=k+1

            

A.to_excel('ProxyA_down.xlsx', index=False)

In [92]:
def star(a):
    if a < 0.01:
        z = '***'
    elif a >=0.01 and a < 0.05:
        z = '**'
    elif a >=0.05 and a < 0.1:
        z = '*'
    else:
        z = ' '
    return z
    

In [93]:
import scipy.stats as stats

AAup=pd.read_excel('ProxyA_up.xlsx', index_col=0) #이것만 있으면 <표1>
AAdown=pd.read_excel('ProxyA_down.xlsx', index_col=0) #이것만 있으면 <표1>
q=['1', '2', '5', '10']

#표2
kk=0
Aup=AAup
Adown=AAdown

Aup1=Aup[Aup['52_WK_HI']>=0.7]
Adown1=Adown[Adown['52_WK_HI']>=0.7]
Aup2=Aup[Aup['52_WK_LO']>=0.7]
Adown2=Adown[Adown['52_WK_LO']>=0.7]
UP=pd.DataFrame()
DOWN=pd.DataFrame()
UP['SR10%(HI>0.7)('+str(len(Aup1))+')data']=0
UP['SR10%(LO>0.7)('+str(len(Aup2))+')data']=0
UP['Difference(SR10%)']=0
DOWN['SR10%(HI>0.7)('+str(len(Adown1))+')data']=0
DOWN['SR10%(LO>0.7)('+str(len(Adown2))+')data']=0
DOWN['Difference(SR10%)']=0
for i in range(0, 4):
    UP.loc["Day"+q[i]]=[star(stats.ttest_1samp(a=Aup1['Day'+q[i]], popmean=0).pvalue)+str(round(100*Aup1['Day'+q[i]].mean(),2))+'('+str(round(100*stats.ttest_1samp(a=Aup1['Day'+q[i]], popmean=0).pvalue,2))+'%)',
                        star(stats.ttest_1samp(a=Aup2['Day'+q[i]], popmean=0).pvalue)+str(round(100*Aup2['Day'+q[i]].mean(),2))+'('+str(round(100*stats.ttest_1samp(a=Aup2['Day'+q[i]], popmean=0).pvalue,2))+'%)',
                        star(stats.ttest_ind(Aup1['Day'+q[i]], Aup2['Day'+q[i]], equal_var=False).pvalue)+str(round(100*(Aup1['Day'+q[i]].mean()-Aup2['Day'+q[i]].mean()),2))+
                        '('+str(round(100*stats.ttest_ind(Aup1['Day'+q[i]], Aup2['Day'+q[i]], equal_var=False).pvalue,2))+'%)']                        
    DOWN.loc["Day"+q[i]]=[star(stats.ttest_1samp(a=Adown1['Day'+q[i]], popmean=0).pvalue)+str(round(100*Adown1['Day'+q[i]].mean(),2))+'('+str(round(100*stats.ttest_1samp(a=Adown1['Day'+q[i]], popmean=0).pvalue,2))+'%)',
                          star(stats.ttest_1samp(a=Adown2['Day'+q[i]], popmean=0).pvalue)+str(round(100*Adown2['Day'+q[i]].mean(),2))+'('+str(round(100*stats.ttest_1samp(a=Adown2['Day'+q[i]], popmean=0).pvalue,2))+'%)',
                          star(stats.ttest_ind(Adown1['Day'+q[i]], Adown2['Day'+q[i]], equal_var=False).pvalue)+str(round(100*(Adown1['Day'+q[i]].mean()-Adown2['Day'+q[i]].mean()),2))+
                          '('+str(round(100*stats.ttest_ind(Adown1['Day'+q[i]], Adown2['Day'+q[i]], equal_var=False).pvalue,2))+'%)']
UP.to_csv('HILO/'+'표2-주가상승-제외기업'+str(kk)+'개.csv')#저장  
DOWN.to_csv('HILO/'+'표2-주가하락-제외기업'+str(kk)+'개.csv')#저장  

#표3,4
for k in range(4,5):
    if(k==3):
        word='CAP'
    if(k==4):
        word='std'
    Aup1.loc[:, word+'_rank'] = Aup1.loc[:, word].rank(axis=0, method='min', ascending=True)
    AAup1=Aup1[Aup1[word+'_rank']>0.7*len(Aup1)]
    AAup2=Aup1[Aup1[word+'_rank']<0.3*len(Aup1)]
    Aup2.loc[:, word+'_rank'] = Aup2.loc[:, word].rank(axis=0, method='min', ascending=True)
    AAup3=Aup2[Aup2[word+'_rank']>0.7*len(Aup2)]
    AAup4=Aup2[Aup2[word+'_rank']<0.3*len(Aup2)]
    Adown1.loc[:, word+'_rank'] = Adown1.loc[:, word].rank(axis=0, method='min', ascending=True)
    AAdown1=Adown1[Adown1[word+'_rank']>0.7*len(Adown1)]
    AAdown2=Adown1[Adown1[word+'_rank']<0.3*len(Adown1)]
    Adown2.loc[:, word+'_rank'] = Adown2.loc[:, word].rank(axis=0, method='min', ascending=True)
    AAdown3=Adown2[Adown2[word+'_rank']>0.7*len(Adown2)]
    AAdown4=Adown2[Adown2[word+'_rank']<0.3*len(Adown2)]
    UP=pd.DataFrame()
    DOWN=pd.DataFrame()
    UP['SR10%(HI>0.7)_CAR('+str(len(AAup1))+'/'+str(len(AAup2))+')data']=0
    UP['SR10%(LO>0.7)_CAR('+str(len(AAup3))+'/'+str(len(AAup4))+')data']=0
    UP['Difference(SR10%)']=0
    DOWN['SR10%(HI>0.7)_CAR('+str(len(AAdown1))+'/'+str(len(AAdown2))+')data']=0
    DOWN['SR10%(LO>0.7)_CAR('+str(len(AAdown3))+'/'+str(len(AAdown4))+')data']=0
    DOWN['Difference(SR10%)']=0
    for i in range(0, 4):
        UP.loc['Day'+q[i]]=[star(stats.ttest_1samp(a=AAup1['Day'+q[i]], popmean=0).pvalue)+str(round(100*AAup1['Day'+q[i]].mean(),2))+'/'
                            +star(stats.ttest_1samp(a=AAup2['Day'+q[i]], popmean=0).pvalue)+str(round(100*AAup2['Day'+q[i]].mean(),2)),
                            star(stats.ttest_1samp(a=AAup3['Day'+q[i]], popmean=0).pvalue)+str(round(100*AAup3['Day'+q[i]].mean(),2))+'/'
                            +star(stats.ttest_1samp(a=AAup4['Day'+q[i]], popmean=0).pvalue)+str(round(100*AAup4['Day'+q[i]].mean(),2)),
                            star(stats.ttest_ind(AAup1['Day'+q[i]], AAup3['Day'+q[i]], equal_var=False).pvalue)+str(round(100*(AAup1['Day'+q[i]].mean()-AAup3['Day'+q[i]].mean()),2))+'/'
                            +star(stats.ttest_ind(AAup2['Day'+q[i]], AAup4['Day'+q[i]], equal_var=False).pvalue)+str(round(100*(AAup2['Day'+q[i]].mean()-AAup4['Day'+q[i]].mean()),2))]
        DOWN.loc['Day'+q[i]]=[star(stats.ttest_1samp(a=AAdown1['Day'+q[i]], popmean=0).pvalue)+str(round(100*AAdown1['Day'+q[i]].mean(),2))+'/'
                            +star(stats.ttest_1samp(a=AAdown2['Day'+q[i]], popmean=0).pvalue)+str(round(100*AAdown2['Day'+q[i]].mean(),2)),
                            star(stats.ttest_1samp(a=AAdown3['Day'+q[i]], popmean=0).pvalue)+str(round(100*AAdown3['Day'+q[i]].mean(),2))+'/'
                            +star(stats.ttest_1samp(a=AAdown4['Day'+q[i]], popmean=0).pvalue)+str(round(100*AAdown4['Day'+q[i]].mean(),2)),
                            star(stats.ttest_ind(AAdown1['Day'+q[i]], AAdown3['Day'+q[i]], equal_var=False).pvalue)+str(round(100*(AAdown1['Day'+q[i]].mean()-AAdown3['Day'+q[i]].mean()),2))+'/'
                            +star(stats.ttest_ind(AAdown2['Day'+q[i]], AAdown4['Day'+q[i]], equal_var=False).pvalue)+str(round(100*(AAdown2['Day'+q[i]].mean()-AAdown4['Day'+q[i]].mean()),2))]
    UP.to_csv('HILO/'+'표'+str(k)+'-주가상승-제외기업'+str(kk)+'개.csv')#저장  
    DOWN.to_csv('HILO/'+'표'+str(k)+'-주가하락-제외기업'+str(kk)+'개.csv')#저장  
    
    
for e in range(5,8):
    if(e==5):
        D='1'
    if(e==6):
        D='5'
    if(e==7):
        D='10'
    X_name=['constant','52_WK_HI','52_WK_LO','CONTRADICTION']
    XAup = Aup.loc[:, X_name[1:4]].values
    Y = Aup.loc[:, 'Day'+D].values
    resAup1 = sm.OLS(Y, sm.add_constant(XAup)).fit()
    XAdown = Adown.loc[:, X_name[1:4]].values
    Y = Adown.loc[:, 'Day'+D].values
    resAdown1 = sm.OLS(Y, sm.add_constant(XAdown)).fit()
    UP=pd.DataFrame()
    DOWN=pd.DataFrame()
    UP['A(diff10%)('+str(len(Aup))+')data']=0
    DOWN['A(diff10%)('+str(len(Adown))+')data']=0
    for i in range(0, len(X_name)):
        UP.loc[X_name[i]]=[star(resAup1.pvalues[i])+str(round(resAup1.params[i],2))+'('+str(round(100*resAup1.pvalues[i],2))+'%)']
        DOWN.loc[X_name[i]]=[star(resAdown1.pvalues[i])+str(round(resAdown1.params[i],2))+'('+str(round(100*resAdown1.pvalues[i],2))+'%)']

    UP.to_csv('HILO/'+'표'+str(e)+'-2-주가상승-제외기업'+str(kk)+'개.csv')#저장  
    DOWN.to_csv('HILO/'+'표'+str(e)+'-2-주가하락-제외기업'+str(kk)+'개.csv')#저장   

In [94]:
Y = Adown.loc[:, 'Day'+D].values
#resAdown1 = sm.OLS(Y, sm.add_constant(XAdown)).fit()

In [95]:
sm.add_constant(XAdown)

array([[1.        , 1.        , 0.08050066, 0.        ],
       [1.        , 0.80645161, 0.675     , 1.        ],
       [1.        , 0.74566474, 0.88372093, 1.        ],
       [1.        , 1.        , 0.18300056, 0.        ],
       [1.        , 0.59817916, 0.98102217, 1.        ],
       [1.        , 1.        , 0.28352113, 0.        ],
       [1.        , 1.        , 0.04766813, 0.        ],
       [1.        , 0.58228756, 0.08186355, 0.        ],
       [1.        , 1.        , 0.44293077, 0.        ],
       [1.        , 0.61091637, 0.8557377 , 0.        ],
       [1.        , 1.        , 0.44594595, 0.        ],
       [1.        , 0.90766998, 0.55756458, 0.        ],
       [1.        , 0.64206642, 0.8683908 , 1.        ],
       [1.        , 1.        , 0.23267554, 0.        ],
       [1.        , 0.57704184, 0.57368735, 1.        ],
       [1.        , 0.44828551, 0.73846154, 1.        ],
       [1.        , 0.36040782, 1.        , 0.        ],
       [1.        , 0.22687624,